In [1]:
#!/usr/local/bin/python3
# assign.py : Assign people to teams
#
# Code by: name IU ID
#
# Based on skeleton code by D. Crandall and B551 Staff, September 2021
#

In [2]:
import sys
import time
import numpy as np

In [3]:
def assign_random_groups():

    if len(people)% 3 ==0:
        poss_n_groups = int(len(people)/3)
    else:
        poss_n_groups = int(np.floor(len(people)/3))+1

    rand_groups = np.array([np.random.randint(0,poss_n_groups+1) for x in range(len(people))])

    n_in_group =[np.count_nonzero(rand_groups == n) for n in range(poss_n_groups+1)]

    while max(n_in_group)>3:
        rand_groups = np.array([np.random.randint(0,poss_n_groups) for x in range(len(people))])
        n_in_group =[np.count_nonzero(rand_groups == n) for n in range(poss_n_groups+1)]

    assigned_groups = []
    [assigned_groups.append(list(people[np.where(rand_groups ==n)])) for n in range(poss_n_groups+1)]

    groups = assigned_groups

    try:
        groups.pop([groups[n]==[]for n in range(len(groups))].index(1))
    except:
        pass

#     print(f'groups {groups}')
    
    return groups

# Cost function

In [4]:
def grading():
    n_groups = len(groups)
    
    return n_groups*5

In [5]:
def wrong_group_size():

    assigned_group_size=[groups_size[group] for group in group_assigned]
    group_size_compare = [assigned_group_size[n]==want_group_size[n] for n in range(len(people))]
    n_wrong_groups = len(people) - np.count_nonzero(group_size_compare)
    
    return n_wrong_groups*2


In [6]:
def wanted_not_in_assigned():

    people_assigned_mask = people_assigned.copy()
    for n, row in enumerate(want):
        for m, item in enumerate(row):
            if item == 'xxx':
                people_assigned_mask[n] = people_assigned_mask[n]+['xxx']

    match_wanted_assigned = [[want[i][n] in people_assigned_mask[i] for n in range(len(want[i]))] for i in range(len(people))]
    match_wanted_assigned = [elem for row in match_wanted_assigned for elem in row]
    
    wanted_not_assigned = len(match_wanted_assigned)- np.count_nonzero(match_wanted_assigned)
    
    return wanted_not_assigned *(0.05*60)

In [7]:
def not_wanted_in_assigned():    
    match_not_want_assigned = [[not_want[i][n] in people_assigned[i] for n in range(len(not_want[i]))] for i in range(len(people))]
    match_not_want_assigned = [elem for row in match_not_want_assigned for elem in row]
    not_want_assigned = np.count_nonzero(match_not_want_assigned)

    return not_want_assigned*10

In [8]:
def calculate_cost():

    cost =int(grading() + wrong_group_size() + wanted_not_in_assigned() + not_wanted_in_assigned())
   
    return cost



# Random search
test to see if the cost functions and assigne random group funcions are coded correctly


In [14]:
time_0 = time.time()

filename = 'test3.txt'
with open(filename) as f:
    lines = f.readlines()

survey_text = [line.rstrip('\n') for line in lines]
survey_data = np.array([entry.split() for entry in survey_text])

#survey inputs
people = survey_data[:,0]
want = [item.split('-') for item in survey_data[:,1]]
not_want = [item.split(',') for item in survey_data[:,2]]
want_group_size = [len(item) for item in want]

min_cost = 100000000
n = 1
for n in range(1000):

    # assign group
    groups = assign_random_groups()
    # #group example from assignment instructions
    # groups = [['djcran','vkvats','nthakurd'], ["sahmaini"], ["sulagaop", 'fanjun']]

    # group stats
    n_groups = len(groups)
    groups_size = [len(group) for group in groups]

    people_group_mat = [[people[x] in group for group in groups] for x in range(len(people))]
    group_assigned = [people_group_mat[n].index(1) for n in range(len(people_group_mat))]

    people_assigned =[groups[n] for n in group_assigned]

    cost = calculate_cost()
    
    if cost < min_cost:
        min_cost_groups = groups
        min_cost = cost
        print(min_cost_groups, min_cost)

print(f'done in {np.round(time.time()-time_0, 3)} seconds')

    
   

[['nthakurd', 'zhou', 'zhang'], ['djcran', 'zhao', 'wang5'], ['fanjun', 'vkvats', 'feng'], ['sulagaop', 'wu'], ['sahmaini', 'sun', 'li'], ['zheng', 'wang', 'chen'], ['qian', 'zhang3', 'li4']] 131
[['zhou', 'wang'], ['djcran', 'fanjun', 'wang5'], ['nthakurd', 'sun', 'li4'], ['sahmaini', 'qian', 'li'], ['zhao', 'zhang3', 'zhang'], ['vkvats', 'feng', 'chen'], ['sulagaop', 'wu', 'zheng']] 130
[['sahmaini', 'wang', 'li4'], ['fanjun', 'zhao', 'wang5'], ['sun', 'li', 'zheng'], ['djcran', 'chen', 'zhang3'], ['qian', 'zhou', 'zhang'], ['nthakurd', 'wu'], ['sulagaop', 'vkvats', 'feng']] 129
[['zhang3', 'li4', 'wang5'], ['sulagaop', 'fanjun', 'wang'], ['zhao', 'feng'], ['sahmaini', 'zhou', 'chen'], ['nthakurd', 'li', 'wu'], ['djcran', 'vkvats', 'zheng'], ['qian', 'sun', 'zhang']] 128
[['djcran', 'zhou', 'zhang'], ['sahmaini', 'vkvats', 'li4'], ['sulagaop', 'li', 'wu'], ['fanjun', 'zhang3', 'wang5'], ['nthakurd', 'zhao', 'zheng'], ['qian', 'sun', 'wang'], ['feng', 'chen']] 127
done in 38.031 secon

In [65]:
def solver(input_file):
    """
    1. This function should take the name of a .txt input file in the format indicated in the assignment.
    2. It should return a dictionary with the following keys:
        - "assigned-groups" : a list of groups assigned by the program, each consisting of usernames separated by hyphens
        - "total-cost" : total cost (time spent by instructors in minutes) in the group assignment
    3. Do not add any extra parameters to the solver() function, or it will break our grading and testing code.
    4. Please do not use any global variables, as it may cause the testing code to fail.
    5. To handle the fact that some problems may take longer than others, and you don't know ahead of time how
       much time it will take to find the best solution, you can compute a series of solutions and then
       call "yield" to return that preliminary solution. Your program can continue yielding multiple times;
       our test program will take the last answer you 'yielded' once time expired.
    """

    # Simple example. First we yield a quick solution
    yield({"assigned-groups": ["vibvats-djcran-zkachwal", "shah12", "vrmath"],
               "total-cost" : 12})

    # Then we think a while and return another solution:
    time.sleep(10)
    yield({"assigned-groups": ["vibvats-djcran-zkachwal", "shah12-vrmath"],
               "total-cost" : 10})

    # This solution will never befound, but that's ok; program will be killed eventually by the
    #  test script.
    while True:
        pass
    
    yield({"assigned-groups": ["vibvats-djcran", "zkachwal-shah12-vrmath"],
               "total-cost" : 9})

In [67]:
solve('test1.txt')

<generator object solver at 0x00000279957C8F20>

In [68]:
for result in solver('test1.txt'):
    print("----- Latest solution:\n" + "\n".join(result["assigned-groups"]))
    print("\nAssignment cost: %d \n" % result["total-cost"])

----- Latest solution:
vibvats-djcran-zkachwal
shah12
vrmath

Assignment cost: 12 



AttributeError: 'builtin_function_or_method' object has no attribute 'sleep'

In [66]:
if __name__ == "__main__":
    if(len(sys.argv) != 2):
        raise(Exception("Error: expected an input filename"))

    for result in solver(sys.argv[1]):
        print("----- Latest solution:\n" + "\n".join(result["assigned-groups"]))
        print("\nAssignment cost: %d \n" % result["total-cost"])
    

Exception: Error: expected an input filename